In [ ]:
"""Plots accuracy figures.
"""

In [2]:
import numpy as np
import glob
import os

In [3]:
from hsi_data import multiversion_matfile_get_field, dset_dims

/scratch0/ilya/locDoc/miniconda2/envs/venvtf/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
%matplotlib ipympl

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import ipympl

In [17]:
plt.style.use('ggplot')

In [5]:
def load_npz_results_from_dirs(res_files):
    """
    Args:
      res_files: list<str>: list of full paths to results.npz files
    Returns:
      dir_results: dict< str, float >: maps mask name to accuracy
    """
    dir_results = {}
    for npz_name in res_files:
        results = np.load(npz_name, allow_pickle=True)['results'].item()
        k = results.keys()[0]
        for k, v in results.iteritems():
            if k in dir_results:
                dir_results[k] = max(dir_results[k], v)
            else:
                dir_results[k] = v
    return dir_results
    

In [172]:
# accumulate
def results_hash_to_mean_std(dir_results, samples=[10,20,50,90], zfill_amount=2):
    """
    Args:
      dir_results: dict< str, float >: maps mask name to accuracy
    """
    dir_y = []
    dir_y_std = []
    #samples = [5,10,20,50]
    for samp in samples:
        if samp < 1:
            samp = int(samp * 10000)
            as_str = '_p%s_' % str(samp).zfill(zfill_amount)
        else:
            as_str = '_s%s_' % str(samp).zfill(zfill_amount)
        valids = [(k,v) for (k,v) in dir_results.iteritems() if as_str in k]
        values_ = np.array([v for (k,v) in valids])
        # make sure values are an array of numbers
        values = []
        for v in values_:
            if not type(v) in [float, np.float, np.float64] and len(v):
                values.append(v[0])
            else:
                values.append(v)
        values = np.array(values)
        
        dir_y.append(values.mean())
        dir_y_std.append(values.std())
    return np.array(dir_y), np.array(dir_y_std)

In [7]:
zfill_amount = 2
samples = [10,20,50,90] # PU SSS
# samples = [5,10,20,50] # KSC SSS
samples = [3,5,10,20] # Dist
# samples = [0.0200,0.0500,0.1000]
samples = [0.005,0.0100,0.0200] # PU hi data
zfill_amount = 4

In [175]:
# paviaU distributed
# samples
samples = [5,10,20] # Dist
zfill_amount = 2
npz_name = '/scratch1/ilya/locDoc/pyfst/june_models/paviau_dist_fst/SVM_results_1983226215.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
fst_means, fst_stds = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)
# dffn
model_type_dir = '/scratch0/ilya/locDoc/pyfst/models/dffn'
res_files = glob.glob(os.path.join(model_type_dir, 'PaviaU_distributed_trainval_s*/*/results.npz'))
dir_results = load_npz_results_from_dirs(res_files)
dffn_means, dffn_stds = results_hash_to_mean_std(dir_results, samples=samples, zfill_amount=zfill_amount)
# eap
model_type_dir = '/scratch0/ilya/locDoc/pyfst/models/eap'
res_files = glob.glob(os.path.join(model_type_dir, 'PaviaU_distributed_trainval_s*/*/results.npz'))
dir_results = load_npz_results_from_dirs(res_files)
eap_means, eap_stds = results_hash_to_mean_std(dir_results, samples=samples, zfill_amount=zfill_amount)
# raw
npz_name = '/scratch0/ilya/locDoc/pyfst/models/pu_raw_svm/SVM_results_6410196752.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
raw_means, raw_stds = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)
# tang
npz_name = '/scratch0/ilya/locDoc/pyfst/models/pu_tang_svm/SVM_results_1051942010.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
tang_means, tang_stds = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)
# gabor
npz_name = '/scratch1/ilya/locDoc/pyfst/june_models/paviau_dist_gabor/SVM_results_5650820553.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
gabor_means, gabor_stds = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)

# percentages
samples = [0.005,0.0100,0.0200] # PU hi data
zfill_amount = 4
npz_name = '/scratch1/ilya/locDoc/pyfst/june_models/paviau_dist_fst/SVM_results_1983226215.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
fst_means2, fst_stds2 = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)
# dffn
model_type_dir = '/scratch0/ilya/locDoc/pyfst/models/dffn'
res_files = glob.glob(os.path.join(model_type_dir, 'PaviaU_distributed_trainval_p*/*/results.npz'))
dir_results = load_npz_results_from_dirs(res_files)
dffn_means2, dffn_stds2 = results_hash_to_mean_std(dir_results, samples=samples, zfill_amount=zfill_amount)
# eap
model_type_dir = '/scratch0/ilya/locDoc/pyfst/models/eap'
res_files = glob.glob(os.path.join(model_type_dir, 'PaviaU_distributed_trainval_p*/*/results.npz'))
dir_results = load_npz_results_from_dirs(res_files)
eap_means2, eap_stds2 = results_hash_to_mean_std(dir_results, samples=samples, zfill_amount=zfill_amount)
# raw
npz_name = '/scratch0/ilya/locDoc/pyfst/models/pu_raw_svm/SVM_results_6410196752.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
raw_means2, raw_stds2 = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)
# tang
npz_name = '/scratch0/ilya/locDoc/pyfst/models/pu_tang_svm/SVM_results_1051942010.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
tang_means2, tang_stds2 = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)
# gabor
npz_name = '/scratch1/ilya/locDoc/pyfst/june_models/paviau_dist_gabor/SVM_results_5650820553.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
gabor_means2, gabor_stds2 = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)


# concat
fst_means = np.concatenate([fst_means, fst_means2])
fst_stds = np.concatenate([fst_stds, fst_stds2])

dffn_means = np.concatenate([dffn_means, dffn_means2])
dffn_stds = np.concatenate([dffn_stds, dffn_stds2])

eap_means = np.concatenate([eap_means, eap_means2])
eap_stds = np.concatenate([eap_stds, eap_stds2])

raw_means = np.concatenate([raw_means, raw_means2])
raw_stds = np.concatenate([raw_stds, raw_stds2])

tang_means = np.concatenate([tang_means, tang_means2])
tang_stds = np.concatenate([tang_stds, tang_stds2])

gabor_means = np.concatenate([gabor_means, gabor_means2])
gabor_stds = np.concatenate([gabor_stds, gabor_stds2])

t = [s_ / (42776 / 9.0) for s_ in [5,10,20]] + [0.005,0.0100,0.0200]
t = [100*s_ for s_ in t]


In [176]:
plt.figure(figsize=[6.4, 4.8])

plt.plot(t, fst_means, 'd-', color='b')
plt.fill_between(t, fst_means-fst_stds, fst_means+fst_stds, facecolor='b', alpha=0.3)

plt.plot(t, dffn_means, 'd-', color='y')
plt.fill_between(t, dffn_means-dffn_stds, dffn_means+dffn_stds, facecolor='y', alpha=0.3)

plt.plot(t, eap_means, 'd-', color='g')
plt.fill_between(t, eap_means-eap_stds, eap_means+eap_stds, facecolor='g', alpha=0.3)

plt.plot(t, raw_means, 'd-', color='purple')
plt.fill_between(t, raw_means-raw_stds, raw_means+raw_stds, facecolor='purple', alpha=0.3)

plt.plot(t, tang_means, 'd-', color='r')
plt.fill_between(t, tang_means-tang_stds, tang_means+tang_stds, facecolor='r', alpha=0.3)

plt.plot(t, gabor_means, 'd-', color='black')
plt.fill_between(t, gabor_means-gabor_stds, gabor_means+gabor_stds, facecolor='black', alpha=0.3)

plt.ylim(ymin=0, ymax=1)
plt.legend(['3D FST', 'DFFN', 'EAP', 'Raw', '3D WST', '3D Gabor'], loc="lower right")
plt.xlabel('Percentage Of Training Data')
plt.ylabel('Overall Accuracy')
plt.title('PaviaU')

plt.savefig('/scratch0/ilya/locDownloads/temp/tgrs_grids/PaviaU_perfs.png', dpi=600)

FigureCanvasNbAgg()

In [83]:
# IP distributed
# samples
samples = [5,10] # Dist
zfill_amount = 2
npz_name = '/scratch1/ilya/locDoc/pyfst/june_models/ip_dist_fst/SVM_results_351309908.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
fst_means, fst_stds = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)
# dffn
model_type_dir = '/scratch0/ilya/locDoc/pyfst/models/dffn'
res_files = glob.glob(os.path.join(model_type_dir, 'IP_distributed_trainval_s*/*/results.npz'))
dir_results = load_npz_results_from_dirs(res_files)
dffn_means, dffn_stds = results_hash_to_mean_std(dir_results, samples=samples, zfill_amount=zfill_amount)
# eap
model_type_dir = '/scratch0/ilya/locDoc/pyfst/models/eap'
res_files = glob.glob(os.path.join(model_type_dir, 'IP_distributed_trainval_s*/*/results.npz'))
dir_results = load_npz_results_from_dirs(res_files)
eap_means, eap_stds = results_hash_to_mean_std(dir_results, samples=samples, zfill_amount=zfill_amount)
# raw
npz_name = '/scratch0/ilya/locDoc/pyfst/models/ip_raw_svm/SVM_results_3322018531.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
raw_means, raw_stds = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)
# tang
npz_name = '/scratch0/ilya/locDoc/pyfst/models/ip_tang_svm/SVM_results_4726895580.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
tang_means, tang_stds = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)
# gabor
npz_name = '/scratch1/ilya/locDoc/pyfst/june_models/ip_dist_gabor/SVM_results_6565639639.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
gabor_means, gabor_stds = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)

# percentages
samples = [0.0200,0.0500,0.1000]
zfill_amount = 4
npz_name = '/scratch1/ilya/locDoc/pyfst/june_models/ip_dist_fst/SVM_results_351309908.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
fst_means2, fst_stds2 = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)
# dffn
model_type_dir = '/scratch0/ilya/locDoc/pyfst/models/dffn'
res_files = glob.glob(os.path.join(model_type_dir, 'IP_distributed_trainval_p*/*/results.npz'))
dir_results = load_npz_results_from_dirs(res_files)
dffn_means2, dffn_stds2 = results_hash_to_mean_std(dir_results, samples=samples, zfill_amount=zfill_amount)
# eap
model_type_dir = '/scratch0/ilya/locDoc/pyfst/models/eap'
res_files = glob.glob(os.path.join(model_type_dir, 'IP_distributed_trainval_p*/*/results.npz'))
dir_results = load_npz_results_from_dirs(res_files)
eap_means2, eap_stds2 = results_hash_to_mean_std(dir_results, samples=samples, zfill_amount=zfill_amount)
# raw
npz_name = '/scratch0/ilya/locDoc/pyfst/models/ip_raw_svm/SVM_results_3322018531.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
raw_means2, raw_stds2 = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)
# tang
npz_name = '/scratch0/ilya/locDoc/pyfst/models/ip_tang_svm/SVM_results_4726895580.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
tang_means2, tang_stds2 = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)
# gabor
npz_name = '/scratch1/ilya/locDoc/pyfst/june_models/ip_dist_gabor/SVM_results_6565639639.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
gabor_means2, gabor_stds2 = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)


# concat
fst_means = np.concatenate([fst_means, fst_means2])
fst_stds = np.concatenate([fst_stds, fst_stds2])

dffn_means = np.concatenate([dffn_means, dffn_means2])
dffn_stds = np.concatenate([dffn_stds, dffn_stds2])

eap_means = np.concatenate([eap_means, eap_means2])
eap_stds = np.concatenate([eap_stds, eap_stds2])

raw_means = np.concatenate([raw_means, raw_means2])
raw_stds = np.concatenate([raw_stds, raw_stds2])

tang_means = np.concatenate([tang_means, tang_means2])
tang_stds = np.concatenate([tang_stds, tang_stds2])

gabor_means = np.concatenate([gabor_means, gabor_means2])
gabor_stds = np.concatenate([gabor_stds, gabor_stds2])

t = [s_ / (10249 / 16.0) for s_ in [5,10]] + [0.0200,0.0500,0.1000]
t = [100*s_ for s_ in t]
# t = samples

found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple


In [84]:
plt.figure()

plt.plot(t, fst_means, 'd-', color='b')
plt.fill_between(t, fst_means-fst_stds, fst_means+fst_stds, facecolor='b', alpha=0.3)

plt.plot(t, dffn_means, 'd-', color='y')
plt.fill_between(t, dffn_means-dffn_stds, dffn_means+dffn_stds, facecolor='y', alpha=0.3)

plt.plot(t, eap_means, 'd-', color='g')
plt.fill_between(t, eap_means-eap_stds, eap_means+eap_stds, facecolor='g', alpha=0.3)

plt.plot(t, raw_means, 'd-', color='purple')
plt.fill_between(t, raw_means-raw_stds, raw_means+raw_stds, facecolor='purple', alpha=0.3)

plt.plot(t, tang_means, 'd-', color='r')
plt.fill_between(t, tang_means-tang_stds, tang_means+tang_stds, facecolor='r', alpha=0.3)

plt.plot(t, gabor_means, 'd-', color='black')
plt.fill_between(t, gabor_means-gabor_stds, gabor_means+gabor_stds, facecolor='black', alpha=0.3)

plt.ylim(ymin=0, ymax=1)
plt.legend(['3D FST', 'DFFN', 'EAP', 'Raw', '3D WST', '3D Gabor'], loc="lower right")
plt.xlabel('Percentage Of Training Data')
plt.ylabel('Overall Accuracy')
plt.title('Indian Pines')

plt.savefig('/scratch0/ilya/locDownloads/temp/tgrs_grids/IP_perfs.png', dpi=600)

FigureCanvasNbAgg()

In [84]:
npz_name = '/scratch0/ilya/locDoc/pyfst/june_models/ksc_sss_fst/SVM_results_3701156503.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)

(array([0.59869726, 0.63241964, 0.66661412, 0.72653581]),
 array([0.04976261, 0.0470072 , 0.05326331, 0.04151317]))

In [114]:
# KSC SSS
# samples
samples = [5,10,20,50] # Dist
zfill_amount = 2
npz_name = '/scratch1/ilya/locDoc/pyfst/june_models/ksc_sss_fst/SVM_results_6403281027.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
fst_means, fst_stds = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)
# # dffn
model_type_dir = '/scratch0/ilya/locDoc/pyfst/models/dffn'
res_files = glob.glob(os.path.join(model_type_dir, 'KSC_strictsinglesite_trainval_s*/*/results.npz'))
dir_results = load_npz_results_from_dirs(res_files)
dffn_means, dffn_stds = results_hash_to_mean_std(dir_results, samples=samples, zfill_amount=zfill_amount)
# # eap
model_type_dir = '/scratch0/ilya/locDoc/pyfst/models/eap'
res_files = glob.glob(os.path.join(model_type_dir, 'KSC_strictsinglesite_trainval_s*/*/results.npz'))
dir_results = load_npz_results_from_dirs(res_files)
eap_means, eap_stds = results_hash_to_mean_std(dir_results, samples=samples, zfill_amount=zfill_amount)
# raw
npz_name = '/scratch0/ilya/locDoc/pyfst/models/KSC_raw_svm/SVM_results_8411790995.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
raw_means, raw_stds = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)
# tang
npz_name = '/scratch0/ilya/locDoc/pyfst/models/KSC_tang_svm/SVM_results_4618754307.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
tang_means, tang_stds = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)
# gabor
npz_name = '/scratch1/ilya/locDoc/pyfst/june_models/ksc_sss_gabor/SVM_results_4069317804.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
gabor_means, gabor_stds = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)

t = samples

found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple


In [115]:
print(' & '.join(['Samples'] + [str(s) for s in samples]) + ' \\\\  \hline')
print(' & '.join(['FST'] + ['%.2f ($\pm$ %.1f)' % (100*m_, 100*s_) for (m_,s_) in zip(fst_means, fst_stds)]) + ' \\\\  \hline')
print(' & '.join(['WST'] + ['%.2f ($\pm$ %.1f)' % (100*m_, 100*s_) for (m_,s_) in zip(tang_means, tang_stds)]) + ' \\\\  \hline')
print(' & '.join(['DFFN'] + ['%.2f ($\pm$ %.1f)' % (100*m_, 100*s_) for (m_,s_) in zip(dffn_means, dffn_stds)]) + ' \\\\  \hline')
print(' & '.join(['EAP'] + ['%.2f ($\pm$ %.1f)' % (100*m_, 100*s_) for (m_,s_) in zip(eap_means, eap_stds)]) + ' \\\\  \hline')
print(' & '.join(['Raw'] + ['%.2f ($\pm$ %.1f)' % (100*m_, 100*s_) for (m_,s_) in zip(raw_means, raw_stds)]) + ' \\\\  \hline')
print(' & '.join(['Gabor'] + ['%.2f ($\pm$ %.1f)' % (100*m_, 100*s_) for (m_,s_) in zip(gabor_means, gabor_stds)]) + ' \\\\  \hline')

Samples & 5 & 10 & 20 & 50 \\  \hline
FST & 62.74 ($\pm$ 4.1) & 65.86 ($\pm$ 4.0) & 71.44 ($\pm$ 3.6) & 75.66 ($\pm$ 1.9) \\  \hline
WST & 53.30 ($\pm$ 5.5) & 59.44 ($\pm$ 4.3) & 65.15 ($\pm$ 4.9) & 68.60 ($\pm$ 4.4) \\  \hline
DFFN & 56.27 ($\pm$ 11.9) & 49.63 ($\pm$ 13.7) & 59.07 ($\pm$ 7.5) & 55.66 ($\pm$ 8.0) \\  \hline
EAP & 56.64 ($\pm$ 6.0) & 61.17 ($\pm$ 5.7) & 64.29 ($\pm$ 2.5) & 61.86 ($\pm$ 4.5) \\  \hline
Raw & 62.79 ($\pm$ 4.3) & 67.61 ($\pm$ 4.1) & 71.58 ($\pm$ 1.8) & 74.61 ($\pm$ 2.8) \\  \hline
Gabor & 63.56 ($\pm$ 5.5) & 65.97 ($\pm$ 4.5) & 70.93 ($\pm$ 5.0) & 72.68 ($\pm$ 2.9) \\  \hline


In [117]:
plt.figure()

plt.plot(t, fst_means, 'd-', color='b')
plt.fill_between(t, fst_means-fst_stds, fst_means+fst_stds, facecolor='b', alpha=0.3)

plt.plot(t, dffn_means, 'd-', color='y')
plt.fill_between(t, dffn_means-dffn_stds, dffn_means+dffn_stds, facecolor='y', alpha=0.3)

plt.plot(t, eap_means, 'd-', color='g')
plt.fill_between(t, eap_means-eap_stds, eap_means+eap_stds, facecolor='g', alpha=0.3)

plt.plot(t, raw_means, 'd-', color='purple')
plt.fill_between(t, raw_means-raw_stds, raw_means+raw_stds, facecolor='purple', alpha=0.3)

plt.plot(t, tang_means, 'd-', color='r')
plt.fill_between(t, tang_means-tang_stds, tang_means+tang_stds, facecolor='r', alpha=0.3)

plt.plot(t, gabor_means, 'd-', color='black')
plt.fill_between(t, gabor_means-gabor_stds, gabor_means+gabor_stds, facecolor='black', alpha=0.3)

plt.ylim(ymin=0, ymax=1)
plt.legend(['3D FST', 'DFFN', 'EAP', 'Raw', '3D WST', '3D Gabor'], loc="lower right")
plt.xlabel('Site Size')
plt.ylabel('Overall Accuracy')
plt.title('KSC - Training from One Site')

plt.savefig('/scratch0/ilya/locDownloads/temp/tgrs_grids/KSC_SSS_perfs.png', dpi=600)

FigureCanvasNbAgg()

In [85]:
# PaviaU SSS
# samples
samples = [10,20,50,90] # Dist
zfill_amount = 2
npz_name = '/scratch1/ilya/locDoc/pyfst/june_models/paviau_sss_fst/SVM_results_880721418.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
fst_means, fst_stds = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)
# # dffn
model_type_dir = '/scratch0/ilya/locDoc/pyfst/models/dffn'
res_files = glob.glob(os.path.join(model_type_dir, 'PaviaU_strictsinglesite_trainval_s*/*/results.npz'))
dir_results = load_npz_results_from_dirs(res_files)
dffn_means, dffn_stds = results_hash_to_mean_std(dir_results, samples=samples, zfill_amount=zfill_amount)
# # eap
model_type_dir = '/scratch0/ilya/locDoc/pyfst/models/eap'
res_files = glob.glob(os.path.join(model_type_dir, 'PaviaU_strictsinglesite_trainval_s*/*/results.npz'))
dir_results = load_npz_results_from_dirs(res_files)
eap_means, eap_stds = results_hash_to_mean_std(dir_results, samples=samples, zfill_amount=zfill_amount)
# raw
npz_name = '/scratch0/ilya/locDoc/pyfst/models/pu_raw_svm/SVM_results_4231370088.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
raw_means, raw_stds = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)
# tang
npz_name = '/scratch0/ilya/locDoc/pyfst/models/pu_tang_svm/SVM_results_899180662.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
tang_means, tang_stds = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)
# gabor
npz_name = '/scratch1/ilya/locDoc/pyfst/june_models/paviau_sss_gabor/SVM_results_3702449457.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
gabor_means, gabor_stds = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)


t = samples

found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple


In [86]:
print(' & '.join(['Samples'] + [str(s) for s in samples]) + ' \\\\  \hline')
print(' & '.join(['FST'] + ['%.2f ($\pm$ %.1f)' % (100*m_, 100*s_) for (m_,s_) in zip(fst_means, fst_stds)]) + ' \\\\  \hline')
print(' & '.join(['WST'] + ['%.2f ($\pm$ %.1f)' % (100*m_, 100*s_) for (m_,s_) in zip(tang_means, tang_stds)]) + ' \\\\  \hline')
print(' & '.join(['DFFN'] + ['%.2f ($\pm$ %.1f)' % (100*m_, 100*s_) for (m_,s_) in zip(dffn_means, dffn_stds)]) + ' \\\\  \hline')
print(' & '.join(['EAP'] + ['%.2f ($\pm$ %.1f)' % (100*m_, 100*s_) for (m_,s_) in zip(eap_means, eap_stds)]) + ' \\\\  \hline')
print(' & '.join(['Raw'] + ['%.2f ($\pm$ %.1f)' % (100*m_, 100*s_) for (m_,s_) in zip(raw_means, raw_stds)]) + ' \\\\  \hline')
print(' & '.join(['Gabor'] + ['%.2f ($\pm$ %.1f)' % (100*m_, 100*s_) for (m_,s_) in zip(gabor_means, gabor_stds)]) + ' \\\\  \hline')

Samples & 10 & 20 & 50 & 90 \\  \hline
FST & 55.59 ($\pm$ 10.3) & 53.31 ($\pm$ 9.6) & 56.35 ($\pm$ 8.4) & 59.33 ($\pm$ 7.4) \\  \hline
WST & 44.53 ($\pm$ 8.7) & 48.01 ($\pm$ 8.1) & 49.48 ($\pm$ 8.5) & 51.59 ($\pm$ 6.6) \\  \hline
DFFN & 43.36 ($\pm$ 14.8) & 41.59 ($\pm$ 16.2) & 49.31 ($\pm$ 10.4) & 48.32 ($\pm$ 6.4) \\  \hline
EAP & 49.29 ($\pm$ 13.1) & 54.84 ($\pm$ 9.9) & 57.31 ($\pm$ 7.5) & 56.81 ($\pm$ 8.7) \\  \hline
Raw & 50.74 ($\pm$ 11.0) & 53.11 ($\pm$ 8.9) & 55.21 ($\pm$ 9.0) & 55.24 ($\pm$ 7.3) \\  \hline
Gabor & 52.96 ($\pm$ 10.2) & 53.33 ($\pm$ 8.9) & 56.97 ($\pm$ 9.2) & 60.00 ($\pm$ 9.7) \\  \hline


In [87]:
plt.figure()

plt.plot(t, fst_means, 'd-', color='b')
plt.fill_between(t, fst_means-fst_stds, fst_means+fst_stds, facecolor='b', alpha=0.3)

plt.plot(t, dffn_means, 'd-', color='y')
plt.fill_between(t, dffn_means-dffn_stds, dffn_means+dffn_stds, facecolor='y', alpha=0.3)

plt.plot(t, eap_means, 'd-', color='g')
plt.fill_between(t, eap_means-eap_stds, eap_means+eap_stds, facecolor='g', alpha=0.3)

plt.plot(t, raw_means, 'd-', color='purple')
plt.fill_between(t, raw_means-raw_stds, raw_means+raw_stds, facecolor='purple', alpha=0.3)

plt.plot(t, tang_means, 'd-', color='r')
plt.fill_between(t, tang_means-tang_stds, tang_means+tang_stds, facecolor='r', alpha=0.3)

plt.plot(t, gabor_means, 'd-', color='black')
plt.fill_between(t, gabor_means-gabor_stds, gabor_means+gabor_stds, facecolor='black', alpha=0.3)

plt.ylim(ymin=0, ymax=1)
plt.legend(['3D FST', 'DFFN', 'EAP', 'Raw', '3D WST', '3D Gabor'], loc="lower right")
plt.xlabel('Site Size')
plt.ylabel('Overall Accuracy')
plt.title('PaviaU - Training from One Site')

plt.savefig('/scratch0/ilya/locDownloads/temp/tgrs_grids/PaviaU_SSS_perfs.png', dpi=600)

FigureCanvasNbAgg()

In [107]:
# Bots SSS
# samples
samples = [3,5,10,20] # Dist
zfill_amount = 2
npz_name = '/scratch1/ilya/locDoc/pyfst/june_models/bots_sss_fst/SVM_results_3535561787.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
fst_means, fst_stds = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)
# # dffn
model_type_dir = '/scratch0/ilya/locDoc/pyfst/models/dffn'
res_files = glob.glob(os.path.join(model_type_dir, 'Botswana_strictsinglesite_trainval_s*/*/results.npz'))
dir_results = load_npz_results_from_dirs(res_files)
dffn_means, dffn_stds = results_hash_to_mean_std(dir_results, samples=samples, zfill_amount=zfill_amount)
# # eap
model_type_dir = '/scratch0/ilya/locDoc/pyfst/models/eap'
res_files = glob.glob(os.path.join(model_type_dir, 'Botswana_strictsinglesite_trainval_s*/*/results.npz'))
dir_results = load_npz_results_from_dirs(res_files)
eap_means, eap_stds = results_hash_to_mean_std(dir_results, samples=samples, zfill_amount=zfill_amount)
# raw
npz_name = '/scratch0/ilya/locDoc/pyfst/models/Botswana_raw_svm/SVM_results_1947462706.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
raw_means, raw_stds = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)
# tang
npz_name = '/scratch0/ilya/locDoc/pyfst/models/Botswana_tang_svm/SVM_results_9172113203.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
tang_means, tang_stds = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)
# gabor
npz_name = '/scratch1/ilya/locDoc/pyfst/june_models/bots_sss_gabor/SVM_results_7690107823.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
gabor_means, gabor_stds = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)

t = samples

found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple


In [108]:
print(' & '.join(['Samples'] + [str(s) for s in samples]) + ' \\\\  \hline')
print(' & '.join(['FST'] + ['%.2f ($\pm$ %.1f)' % (100*m_, 100*s_) for (m_,s_) in zip(fst_means, fst_stds)]) + ' \\\\  \hline')
print(' & '.join(['WST'] + ['%.2f ($\pm$ %.1f)' % (100*m_, 100*s_) for (m_,s_) in zip(tang_means, tang_stds)]) + ' \\\\  \hline')
print(' & '.join(['DFFN'] + ['%.2f ($\pm$ %.1f)' % (100*m_, 100*s_) for (m_,s_) in zip(dffn_means, dffn_stds)]) + ' \\\\  \hline')
print(' & '.join(['EAP'] + ['%.2f ($\pm$ %.1f)' % (100*m_, 100*s_) for (m_,s_) in zip(eap_means, eap_stds)]) + ' \\\\  \hline')
print(' & '.join(['Raw'] + ['%.2f ($\pm$ %.1f)' % (100*m_, 100*s_) for (m_,s_) in zip(raw_means, raw_stds)]) + ' \\\\  \hline')
print(' & '.join(['Gabor'] + ['%.2f ($\pm$ %.1f)' % (100*m_, 100*s_) for (m_,s_) in zip(gabor_means, gabor_stds)]) + ' \\\\  \hline')

Samples & 3 & 5 & 10 & 20 \\  \hline
FST & 76.42 ($\pm$ 6.8) & 77.56 ($\pm$ 4.8) & 79.79 ($\pm$ 4.1) & 85.80 ($\pm$ 2.6) \\  \hline
WST & 52.46 ($\pm$ 4.9) & 56.26 ($\pm$ 4.0) & 60.40 ($\pm$ 3.9) & 68.90 ($\pm$ 6.1) \\  \hline
DFFN & 32.96 ($\pm$ 12.0) & 38.34 ($\pm$ 10.6) & 40.81 ($\pm$ 11.1) & 38.92 ($\pm$ 4.2) \\  \hline
EAP & 66.43 ($\pm$ 13.1) & 72.12 ($\pm$ 5.4) & 72.25 ($\pm$ 5.8) & 74.68 ($\pm$ 6.4) \\  \hline
Raw & 70.98 ($\pm$ 4.0) & 69.52 ($\pm$ 4.2) & 75.67 ($\pm$ 2.6) & 80.71 ($\pm$ 2.1) \\  \hline
Gabor & 75.17 ($\pm$ 5.8) & 75.78 ($\pm$ 3.9) & 79.11 ($\pm$ 4.1) & 84.83 ($\pm$ 2.6) \\  \hline


In [110]:
plt.figure()

plt.plot(t, fst_means, 'd-', color='b')
plt.fill_between(t, fst_means-fst_stds, fst_means+fst_stds, facecolor='b', alpha=0.3)

plt.plot(t, dffn_means, 'd-', color='y')
plt.fill_between(t, dffn_means-dffn_stds, dffn_means+dffn_stds, facecolor='y', alpha=0.3)

plt.plot(t, eap_means, 'd-', color='g')
plt.fill_between(t, eap_means-eap_stds, eap_means+eap_stds, facecolor='g', alpha=0.3)

plt.plot(t, raw_means, 'd-', color='purple')
plt.fill_between(t, raw_means-raw_stds, raw_means+raw_stds, facecolor='purple', alpha=0.3)

plt.plot(t, tang_means, 'd-', color='r')
plt.fill_between(t, tang_means-tang_stds, tang_means+tang_stds, facecolor='r', alpha=0.3)

plt.plot(t, gabor_means, 'd-', color='black')
plt.fill_between(t, gabor_means-gabor_stds, gabor_means+gabor_stds, facecolor='black', alpha=0.3)

plt.ylim(ymin=0, ymax=1)
plt.legend(['3D FST', 'DFFN', 'EAP', 'Raw', '3D WST', '3D Gabor'], loc="lower right")
plt.title('Botswana - Training from One Site')
plt.ylabel('Overall Accuracy')
plt.xlabel('Site Size')

plt.savefig('/scratch0/ilya/locDownloads/temp/tgrs_grids/Botswana_SSS_perfs.png', dpi=600)

FigureCanvasNbAgg()

In [144]:
# IP SSS
# samples
samples = [5,10] # Dist
zfill_amount = 2
npz_name = '/scratch1/ilya/locDoc/pyfst/june_models/ip_sss_fst/SVM_results_1620725352.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
fst_means, fst_stds = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)
# dffn
model_type_dir = '/scratch1/ilya/locDoc/pyfst/june_models/dffn/ip/dffn'
res_files = glob.glob(os.path.join(model_type_dir, 'IP_strictsinglesite_trainval_s*/*/results.npz'))
dir_results = load_npz_results_from_dirs(res_files)
dffn_means, dffn_stds = results_hash_to_mean_std(dir_results, samples=samples, zfill_amount=zfill_amount)
# # eap
model_type_dir = '/scratch1/ilya/locDoc/pyfst/june_models/eap/ip/eap'
res_files = glob.glob(os.path.join(model_type_dir, 'IP_strictsinglesite_trainval_s*/*/results.npz'))
dir_results = load_npz_results_from_dirs(res_files)
eap_means, eap_stds = results_hash_to_mean_std(dir_results, samples=samples, zfill_amount=zfill_amount)
# raw
npz_name = '/scratch0/ilya/locDoc/pyfst/june_models/IP_raw_svm/SVM_results_2495889255.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
raw_means, raw_stds = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)
# tang
npz_name = '/scratch1/ilya/locDoc/pyfst/june_models/IP_wst/SVM_results_4206824447.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
tang_means, tang_stds = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)
# gabor
npz_name = '/scratch1/ilya/locDoc/pyfst/june_models/ip_sss_gabor/SVM_results_9445987777.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
gabor_means, gabor_stds = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)

# percentages
samples = [0.0200,0.0500,0.1000]
zfill_amount = 4
npz_name = '/scratch1/ilya/locDoc/pyfst/june_models/ip_sss_fst/SVM_results_1620725352.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
fst_means2, fst_stds2 = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)
# dffn
model_type_dir = '/scratch1/ilya/locDoc/pyfst/june_models/dffn/ip/dffn'
res_files = glob.glob(os.path.join(model_type_dir, 'IP_strictsinglesite_trainval_p*/*/results.npz'))
dir_results = load_npz_results_from_dirs(res_files)
dffn_means2, dffn_stds2 = results_hash_to_mean_std(dir_results, samples=samples, zfill_amount=zfill_amount)
# # eap
model_type_dir = '/scratch1/ilya/locDoc/pyfst/june_models/eap/ip/eap'
res_files = glob.glob(os.path.join(model_type_dir, 'IP_strictsinglesite_trainval_p*/*/results.npz'))
dir_results = load_npz_results_from_dirs(res_files)
eap_means2, eap_stds2 = results_hash_to_mean_std(dir_results, samples=samples, zfill_amount=zfill_amount)
# raw
npz_name = '/scratch0/ilya/locDoc/pyfst/june_models/IP_raw_svm/SVM_results_2495889255.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
raw_means2, raw_stds2 = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)
# tang
npz_name = '/scratch1/ilya/locDoc/pyfst/june_models/IP_wst/SVM_results_4206824447.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
tang_means2, tang_stds2 = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)
# gabor
npz_name = '/scratch1/ilya/locDoc/pyfst/june_models/ip_sss_gabor/SVM_results_9445987777.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
gabor_means2, gabor_stds2 = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)


# concat
fst_means = np.concatenate([fst_means, fst_means2])
fst_stds = np.concatenate([fst_stds, fst_stds2])

dffn_means = np.concatenate([dffn_means, dffn_means2])
dffn_stds = np.concatenate([dffn_stds, dffn_stds2])

eap_means = np.concatenate([eap_means, eap_means2])
eap_stds = np.concatenate([eap_stds, eap_stds2])

raw_means = np.concatenate([raw_means, raw_means2])
raw_stds = np.concatenate([raw_stds, raw_stds2])

tang_means = np.concatenate([tang_means, tang_means2])
tang_stds = np.concatenate([tang_stds, tang_stds2])

gabor_means = np.concatenate([gabor_means, gabor_means2])
gabor_stds = np.concatenate([gabor_stds, gabor_stds2])

t = [s_ / (10249 / 16.0) for s_ in [5,10]] + [0.0200,0.0500,0.1000]
t = [100*s_ for s_ in t]
# t = samples

found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple


In [145]:
plt.figure()

plt.plot(t, fst_means, 'd-', color='b')
plt.fill_between(t, fst_means-fst_stds, fst_means+fst_stds, facecolor='b', alpha=0.3)

plt.plot(t, dffn_means, 'd-', color='y')
plt.fill_between(t, dffn_means-dffn_stds, dffn_means+dffn_stds, facecolor='y', alpha=0.3)

plt.plot(t, eap_means, 'd-', color='g')
plt.fill_between(t, eap_means-eap_stds, eap_means+eap_stds, facecolor='g', alpha=0.3)

plt.plot(t, raw_means, 'd-', color='purple')
plt.fill_between(t, raw_means-raw_stds, raw_means+raw_stds, facecolor='purple', alpha=0.3)

plt.plot(t, tang_means, 'd-', color='r')
plt.fill_between(t, tang_means-tang_stds, tang_means+tang_stds, facecolor='r', alpha=0.3)

plt.plot(t, gabor_means, 'd-', color='black')
plt.fill_between(t, gabor_means-gabor_stds, gabor_means+gabor_stds, facecolor='black', alpha=0.3)

plt.ylim(ymin=0, ymax=1)
plt.legend(['3D FST', 'DFFN', 'EAP', 'Raw', '3D WST', '3D Gabor'], loc="lower right")
# plt.legend(['3D FST', 'EAP', 'Raw', '3D WST', '3D Gabor'], loc="lower right")
plt.xlabel('Site Size (as Percentage Of Training Data)')
plt.ylabel('Overall Accuracy')
plt.title('Indian Pines - Training from One Site')

plt.savefig('/scratch0/ilya/locDownloads/temp/tgrs_grids/IP_SSS_perfs.png', dpi=600)

FigureCanvasNbAgg()

In [150]:
# KSC dist
# samples
samples = [5,10,20,50] # Dist
zfill_amount = 2
npz_name = '/scratch1/ilya/locDoc/pyfst/june_models/ksc_dist_fst/SVM_results_3378641093.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
fst_means, fst_stds = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)
# dffn
model_type_dir = '/scratch1/ilya/locDoc/pyfst/june_models/dffn/dffn'
res_files = glob.glob(os.path.join(model_type_dir, 'KSC_distributed_trainval_s*/*/results.npz'))
dir_results = load_npz_results_from_dirs(res_files)
dffn_means, dffn_stds = results_hash_to_mean_std(dir_results, samples=samples, zfill_amount=zfill_amount)
# # eap
model_type_dir = '/scratch1/ilya/locDoc/pyfst/june_models/eap/eap'
res_files = glob.glob(os.path.join(model_type_dir, 'KSC_distributed_trainval_s*/*/results.npz'))
dir_results = load_npz_results_from_dirs(res_files)
eap_means, eap_stds = results_hash_to_mean_std(dir_results, samples=samples, zfill_amount=zfill_amount)
# raw
npz_name = '/scratch0/ilya/locDoc/pyfst/june_models/KSC_raw_svm/SVM_results_354804614.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
raw_means, raw_stds = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)
# tang
npz_name = '/scratch1/ilya/locDoc/pyfst/june_models/KSC_wst/SVM_results_1732515566.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
tang_means, tang_stds = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)
# gabor
npz_name = '/scratch1/ilya/locDoc/pyfst/june_models/ksc_dist_gabor/SVM_results_2540158683.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
gabor_means, gabor_stds = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)

t = samples

found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple


In [151]:
print(' & '.join(['Samples'] + [str(s) for s in samples]) + ' \\\\  \hline')
print(' & '.join(['FST'] + ['%.2f ($\pm$ %.1f)' % (100*m_, 100*s_) for (m_,s_) in zip(fst_means, fst_stds)]) + ' \\\\  \hline')
print(' & '.join(['WST'] + ['%.2f ($\pm$ %.1f)' % (100*m_, 100*s_) for (m_,s_) in zip(tang_means, tang_stds)]) + ' \\\\  \hline')
print(' & '.join(['DFFN'] + ['%.2f ($\pm$ %.1f)' % (100*m_, 100*s_) for (m_,s_) in zip(dffn_means, dffn_stds)]) + ' \\\\  \hline')
print(' & '.join(['EAP'] + ['%.2f ($\pm$ %.1f)' % (100*m_, 100*s_) for (m_,s_) in zip(eap_means, eap_stds)]) + ' \\\\  \hline')
print(' & '.join(['Raw'] + ['%.2f ($\pm$ %.1f)' % (100*m_, 100*s_) for (m_,s_) in zip(raw_means, raw_stds)]) + ' \\\\  \hline')
print(' & '.join(['Gabor'] + ['%.2f ($\pm$ %.1f)' % (100*m_, 100*s_) for (m_,s_) in zip(gabor_means, gabor_stds)]) + ' \\\\  \hline')

Samples & 5 & 10 & 20 & 50 \\  \hline
FST & 86.40 ($\pm$ 2.8) & 93.18 ($\pm$ 1.5) & 97.69 ($\pm$ 0.8) & 99.68 ($\pm$ 0.1) \\  \hline
WST & 82.23 ($\pm$ 2.9) & 90.06 ($\pm$ 1.1) & 95.98 ($\pm$ 1.3) & 99.16 ($\pm$ 0.2) \\  \hline
DFFN & 76.51 ($\pm$ 5.1) & 89.87 ($\pm$ 3.0) & 95.94 ($\pm$ 1.5) & 98.47 ($\pm$ 1.0) \\  \hline
EAP & 64.17 ($\pm$ 7.1) & 68.47 ($\pm$ 5.0) & 74.91 ($\pm$ 2.3) & 81.47 ($\pm$ 2.7) \\  \hline
Raw & 78.25 ($\pm$ 2.1) & 84.36 ($\pm$ 1.8) & 88.97 ($\pm$ 0.6) & 92.56 ($\pm$ 0.6) \\  \hline
Gabor & 83.16 ($\pm$ 2.2) & 89.92 ($\pm$ 1.5) & 95.01 ($\pm$ 0.8) & 98.73 ($\pm$ 0.4) \\  \hline


In [121]:
plt.figure()

plt.plot(t, fst_means, 'd-', color='b')
plt.fill_between(t, fst_means-fst_stds, fst_means+fst_stds, facecolor='b', alpha=0.3)

plt.plot(t, dffn_means, 'd-', color='y')
plt.fill_between(t, dffn_means-dffn_stds, dffn_means+dffn_stds, facecolor='y', alpha=0.3)

plt.plot(t, eap_means, 'd-', color='g')
plt.fill_between(t, eap_means-eap_stds, eap_means+eap_stds, facecolor='g', alpha=0.3)

plt.plot(t, raw_means, 'd-', color='purple')
plt.fill_between(t, raw_means-raw_stds, raw_means+raw_stds, facecolor='purple', alpha=0.3)

plt.plot(t, tang_means, 'd-', color='r')
plt.fill_between(t, tang_means-tang_stds, tang_means+tang_stds, facecolor='r', alpha=0.3)

plt.plot(t, gabor_means, 'd-', color='black')
plt.fill_between(t, gabor_means-gabor_stds, gabor_means+gabor_stds, facecolor='black', alpha=0.3)

plt.ylim(ymin=0, ymax=1.025)
plt.legend(['3D FST', 'DFFN', 'EAP', 'Raw', '3D WST', '3D Gabor'], loc="lower right")
plt.xlabel('Number Of Training Samples')
plt.ylabel('Overall Accuracy')
plt.title('KSC')

plt.savefig('/scratch0/ilya/locDownloads/temp/tgrs_grids/KSC_perfs.png', dpi=600)

FigureCanvasNbAgg()

In [152]:
# Bots dist
# samples
samples = [3,5,10,20] # Dist
zfill_amount = 2
npz_name = '/scratch1/ilya/locDoc/pyfst/june_models/bots_dist_fst/SVM_results_5664262905.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
fst_means, fst_stds = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)
# # dffn
model_type_dir = '/scratch1/ilya/locDoc/pyfst/june_models/dffn/dffn'
res_files = glob.glob(os.path.join(model_type_dir, 'Botswana_distributed_trainval_s*/*/results.npz'))
dir_results = load_npz_results_from_dirs(res_files)
dffn_means, dffn_stds = results_hash_to_mean_std(dir_results, samples=samples, zfill_amount=zfill_amount)
# # eap
model_type_dir = '/scratch0/ilya/locDoc/pyfst/june_models/eap'
res_files = glob.glob(os.path.join(model_type_dir, 'Botswana_distributed_trainval_s*/*/results.npz'))
dir_results = load_npz_results_from_dirs(res_files)
eap_means, eap_stds = results_hash_to_mean_std(dir_results, samples=samples, zfill_amount=zfill_amount)
# raw
npz_name = '/scratch0/ilya/locDoc/pyfst/june_models/Botswana_raw_svm/SVM_results_8584562831.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
raw_means, raw_stds = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)
# tang
npz_name = '/scratch1/ilya/locDoc/pyfst/june_models/Botswana_wst/SVM_results_4549174811.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
tang_means, tang_stds = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)
# gabor
npz_name = '/scratch1/ilya/locDoc/pyfst/june_models/bots_dist_gabor/SVM_results_3348049280.npz'
results = np.load(npz_name, allow_pickle=True)['results'].item()
gabor_means, gabor_stds = results_hash_to_mean_std(results, samples=samples, zfill_amount=zfill_amount)

t = samples

found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple
found tuple


In [153]:
print(' & '.join(['Samples'] + [str(s) for s in samples]) + ' \\\\  \hline')
print(' & '.join(['FST'] + ['%.2f ($\pm$ %.1f)' % (100*m_, 100*s_) for (m_,s_) in zip(fst_means, fst_stds)]) + ' \\\\  \hline')
print(' & '.join(['WST'] + ['%.2f ($\pm$ %.1f)' % (100*m_, 100*s_) for (m_,s_) in zip(tang_means, tang_stds)]) + ' \\\\  \hline')
print(' & '.join(['DFFN'] + ['%.2f ($\pm$ %.1f)' % (100*m_, 100*s_) for (m_,s_) in zip(dffn_means, dffn_stds)]) + ' \\\\  \hline')
print(' & '.join(['EAP'] + ['%.2f ($\pm$ %.1f)' % (100*m_, 100*s_) for (m_,s_) in zip(eap_means, eap_stds)]) + ' \\\\  \hline')
print(' & '.join(['Raw'] + ['%.2f ($\pm$ %.1f)' % (100*m_, 100*s_) for (m_,s_) in zip(raw_means, raw_stds)]) + ' \\\\  \hline')
print(' & '.join(['Gabor'] + ['%.2f ($\pm$ %.1f)' % (100*m_, 100*s_) for (m_,s_) in zip(gabor_means, gabor_stds)]) + ' \\\\  \hline')

Samples & 3 & 5 & 10 & 20 \\  \hline
FST & 89.12 ($\pm$ 2.0) & 91.83 ($\pm$ 1.9) & 96.68 ($\pm$ 1.0) & 98.62 ($\pm$ 0.6) \\  \hline
WST & 79.16 ($\pm$ 3.0) & 87.50 ($\pm$ 4.5) & 94.49 ($\pm$ 1.1) & 97.06 ($\pm$ 1.0) \\  \hline
DFFN & 68.27 ($\pm$ 12.9) & 77.83 ($\pm$ 8.1) & 87.47 ($\pm$ 4.4) & 95.11 ($\pm$ 1.2) \\  \hline
EAP & 57.09 ($\pm$ 29.7) & 80.34 ($\pm$ 7.4) & 87.23 ($\pm$ 2.5) & 92.53 ($\pm$ 3.2) \\  \hline
Raw & 77.53 ($\pm$ 3.6) & 80.91 ($\pm$ 2.0) & 85.10 ($\pm$ 1.8) & 88.85 ($\pm$ 0.7) \\  \hline
Gabor & 85.72 ($\pm$ 1.7) & 90.25 ($\pm$ 2.7) & 95.49 ($\pm$ 1.1) & 97.58 ($\pm$ 0.5) \\  \hline


In [113]:
plt.figure()

plt.plot(t, fst_means, 'd-', color='b')
plt.fill_between(t, fst_means-fst_stds, fst_means+fst_stds, facecolor='b', alpha=0.3)

plt.plot(t, dffn_means, 'd-', color='y')
plt.fill_between(t, dffn_means-dffn_stds, dffn_means+dffn_stds, facecolor='y', alpha=0.3)

plt.plot(t, eap_means, 'd-', color='g')
plt.fill_between(t, eap_means-eap_stds, eap_means+eap_stds, facecolor='g', alpha=0.3)

plt.plot(t, raw_means, 'd-', color='purple')
plt.fill_between(t, raw_means-raw_stds, raw_means+raw_stds, facecolor='purple', alpha=0.3)

plt.plot(t, tang_means, 'd-', color='r')
plt.fill_between(t, tang_means-tang_stds, tang_means+tang_stds, facecolor='r', alpha=0.3)

plt.plot(t, gabor_means, 'd-', color='black')
plt.fill_between(t, gabor_means-gabor_stds, gabor_means+gabor_stds, facecolor='black', alpha=0.3)

plt.ylim(ymin=0, ymax=1.025)
plt.legend(['3D FST', 'DFFN', 'EAP', 'Raw', '3D WST', '3D Gabor'], loc="lower right")
plt.title('Botswana')
plt.ylabel('Overall Accuracy')
plt.xlabel('Number Of Training Samples')

plt.savefig('/scratch0/ilya/locDownloads/temp/tgrs_grids/Botswana_perfs.png', dpi=600)

FigureCanvasNbAgg()

In [65]:
samples = [50] # Dist
zfill_amount = 2
npz_name = '/scratch0/ilya/locDoc/pyfst/june_models/ksc_fst_svm/SVM_results_7105350443.npz'
results_fst = np.load(npz_name, allow_pickle=True)['results'].item()
# fst_means, fst_stds = results_hash_to_mean_std(results_fst, samples=samples, zfill_amount=zfill_amount)
# raw
npz_name = '/scratch0/ilya/locDoc/pyfst/june_models/ksc_sss_gabor_svm/SVM_results_6223977032.npz'
results_raw = np.load(npz_name, allow_pickle=True)['results'].item()
# raw_means, raw_stds = results_hash_to_mean_std(results_raw, samples=samples, zfill_amount=zfill_amount)

In [62]:
names = [k for k, v in results_fst.items()]
arr = np.array([v[0] for k, v in results_fst.items()])
print('%.4f +/- %.4f' %(arr.mean(), arr.std()))

0.7563 +/- 0.0317


In [63]:
idxs = np.argsort(arr)
arr[idxs[0]]
names[idxs[0]]

'/cfarhomes/ilyak/ilyakavalerov@gmail.com/ramawks69/pyfst/masks/KSC_strictsinglesite_trainval_s50_9_307364.mat'

In [66]:
arr = np.array([v[0] for v in results_raw.values()])
print('%.4f +/- %.4f' %(arr.mean(), arr.std()))

0.7174 +/- 0.0301


In [38]:
np.argsort(arr)

array([4, 6, 3, 9, 2, 7, 1, 8, 5, 0])

In [46]:
results_raw

{'/cfarhomes/ilyak/ilyakavalerov@gmail.com/ramawks69/pyfst/masks/KSC_strictsinglesite_trainval_s50_0_665848.mat': (0.7465468099101075,
  0.6905851227182196,
  0.7161611560197501),
 '/cfarhomes/ilyak/ilyakavalerov@gmail.com/ramawks69/pyfst/masks/KSC_strictsinglesite_trainval_s50_1_783796.mat': (0.7513703135277352,
  0.7108108484290642,
  0.7234463563064221),
 '/cfarhomes/ilyak/ilyakavalerov@gmail.com/ramawks69/pyfst/masks/KSC_strictsinglesite_trainval_s50_2_455308.mat': (0.7114667836000877,
  0.6833432341927398,
  0.6787416844380036),
 '/cfarhomes/ilyak/ilyakavalerov@gmail.com/ramawks69/pyfst/masks/KSC_strictsinglesite_trainval_s50_3_643924.mat': (0.7702258276693708,
  0.747084657742553,
  0.7435674013463986),
 '/cfarhomes/ilyak/ilyakavalerov@gmail.com/ramawks69/pyfst/masks/KSC_strictsinglesite_trainval_s50_4_514244.mat': (0.7860118395088797,
  0.767013651889459,
  0.760804044402019),
 '/cfarhomes/ilyak/ilyakavalerov@gmail.com/ramawks69/pyfst/masks/KSC_strictsinglesite_trainval_s50_5_76